# Project 3 - Constructing a database using API and Webscrapping

# Importing libraries

In [6]:
# Required Liberaries:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
from tqdm.auto import tqdm
import sqlalchemy as db

# Functions

In [4]:
def get_response(url):
    response = requests.get(url, headers = headers)
    return response



# USING API

![Twitch Logo](https://theroguewolfe.files.wordpress.com/2014/08/twitchlogo.png)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Twitch** is a video live streaming service operated by Twitch Interactive, a subsidiary of Amazon. Introduced in June 2011 as a spin-off of the general-interest streaming platform, Justin.tv, the site primarily focuses on video game live streaming, including broadcasts of eSports competitions, in addition to music broadcasts, creative content, and more recently, "in real life" streams. Content on the site can be viewed either live or via video on demand.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;I decided to use **Twitch Api** to search for data in this project, because reading its documentation it seemed simple to get access and its practical endpoints to work with. And the access is really intuitive! But the data is limited to some endpoint options, I tried to prioritize some information that can be used later relating to other data.

## Creating tables

### Games, Viewers and Channels

This table will show the games being played the most and their respective channel numbers.

In [9]:
headers = {'Client-ID' : 'h8zvs5dtl6ukn16cvf8s2ade7xw6km', 'Accept' : 'application/vnd.twitchtv.v5+json'}

gameviewers = pd.DataFrame()
for i in range(0,11):
    response = get_response('https://api.twitch.tv/kraken/games/top?offset='+str(i) +'&Limit=100')
    games = response.json()['top']
    
    for game in games:
        today = datetime.date.today()
        game_id = game['game']['_id']
        game_name = game['game']['name']
        viewers = int(game['viewers'])
        channels = int(game['channels'])
        mini_df = pd.DataFrame({'date':[today],
                                'id':[game_id],
                               'name': [game_name],
                               'viewers': [viewers],
                               'channels': [channels]})
        gameviewers = pd.concat([gameviewers,mini_df])

gameviewers = gameviewers.reset_index(drop=True)

In [10]:
gameviewers.head(20)

,date,id,name,viewers,channels
0,2020-03-24,32399,Counter-Strike: Global Offensive,358392,3843
1,2020-03-24,512710,Call of Duty: Modern Warfare,267030,10510
2,2020-03-24,509658,Just Chatting,247930,3412
3,2020-03-24,21779,League of Legends,184497,5943
4,2020-03-24,509538,Animal Crossing: New Horizons,135982,1986
5,2020-03-24,32982,Grand Theft Auto V,123616,2612
6,2020-03-24,33214,Fortnite,122265,11411
7,2020-03-24,515195,Half-Life: Alyx,99175,855
8,2020-03-24,29595,Dota 2,93245,1256
9,2020-03-24,18122,World of Warcraft,52486,2304


### Most viewed channels of top games

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Using the endpoint of the previous table, we take the most watched games and search the channel endpoint to find out which channels are transmitting and receiving so many views.

In [11]:
stream_channel = pd.DataFrame()

for i in tqdm(range(10)):
    response = get_response('https://api.twitch.tv/kraken/games/top')
    
    games = response.json()
    top10games = (games['top'][i]['game']['name'])
    url_query = 'https://api.twitch.tv/kraken/search/streams?query='+top10games+'&Limit=100'

    response = requests.get(url_query, headers = headers)
    streams = response.json()['streams']
    
    for i in range(10):
        try:
            today = datetime.date.today()
            stream_id = streams[i]['channel']['_id']
            stream_name = streams[i]['channel']['name']
            stream_game = streams[i]['game']
            stream_viewers = int(streams[i]['viewers'])
            stream_lang = streams[i]['channel']['broadcaster_language']
            stream_followers = int(streams[i]['channel']['followers'])
            stream_views = int(streams[i]['channel']['views'])
            mini_df = pd.DataFrame({'date':[today],
                                    'streamer_id':[stream_id],
                                    'streamer_name': [stream_name],
                                    'game_name': [stream_game],
                                    'viewers': [stream_viewers],
                                    'language': [stream_lang],
                                    'followers': [stream_followers],
                                    'views': [stream_views]})
            stream_channel = pd.concat([stream_channel,mini_df])
        except:
            pass
stream_channel = stream_channel.reset_index(drop=True)

In [12]:
stream_channel = stream_channel.sort_values(by = 'viewers', ascending = False)
stream_channel.head(20).reset_index()

,index,date,streamer_id,streamer_name,game_name,viewers,language,followers,views
0,0,2020-03-24,31239503,esl_csgo,Counter-Strike: Global Offensive,118128,en,3610976,444244856
1,1,2020-03-24,213762816,csgomc_ru2,Counter-Strike: Global Offensive,89606,ru,52530,1851482
2,2,2020-03-24,213748641,csgomc_ru,Counter-Strike: Global Offensive,50357,ru,319966,21894464
3,20,2020-03-24,40063341,domingo,Just Chatting,38700,fr,724768,65071492
4,76,2020-03-24,23161357,lirik,Half-Life: Alyx,38370,en,2564843,336280993
5,10,2020-03-24,45044816,montanablack88,Call of Duty: Modern Warfare,32827,de,2339265,63482363
6,11,2020-03-24,17337557,drdisrespect,Call of Duty: Modern Warfare,29923,en,4124867,166890407
7,3,2020-03-24,181077473,gaules,Counter-Strike: Global Offensive,28406,pt,989913,97909759
8,30,2020-03-24,51496027,loltyler1,League of Legends,21651,en,3152000,153991000
9,68,2020-03-24,67794893,weplayesport_en,Dota 2,18802,en,111209,11220586


# WEB SCRAPING

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;For Web Scraping I decide to search for data on **Steam** and **Metacrit**, they are sites that are reference in the area and contain information that may be useful in the future.

## Getting data from Steam

![Steam logo](http://icons.iconarchive.com/icons/martz90/circle/128/steam-icon.png)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Steam is a video game digital distribution service by Valve. It was launched as a standalone software client in September 2003 as a way for Valve to provide automatic updates for their games, and expanded to include games from third-party publishers. Steam has also expanded into an online web-based and mobile digital storefront. Steam offers digital rights management (DRM), matchmaking servers, video streaming, and social networking services. It also provides the user with installation and automatic updating of games, and community features such as friends lists and groups, cloud saving, and in-game voice and chat functionality.

### Getting the top 50 selled games on Steam

In [54]:
url = 'https://store.steampowered.com/search/?filter=topsellers'
html = requests.get(url).content
soup = BeautifulSoup(html)

mostselledgames = [games.text.strip().replace('\n','') for games in soup.find_all('div', attrs = {'class': 'col search_name ellipsis'})]

datereleased = [date.text for date in soup.find_all('div', attrs = {'class': 'col search_released responsive_secondrow'})]

try:
    discount = [discount.text.strip().replace('-','').replace('%','') for discount in soup.find_all('div', attrs = {'class': 'col search_discount responsive_secondrow'})]
except:
    discount = [discount.text.strip() for discount in soup.find_all('div', attrs = {'class': 'col search_discount responsive_secondrow'})]

try:
    price = [float(price.text.strip().split()[-1].replace(',','.')) for price in soup.find_all('div', attrs = {'class': 'col search_price_discount_combined responsive_secondrow'})]
except:
    price = [price.text.strip().split()[-1].replace(',','.') for price in soup.find_all('div', attrs = {'class': 'col search_price_discount_combined responsive_secondrow'})]
date_list = [datetime.date.today() for i in range(len(mostselledgames))]
data = list(zip(date_list, mostselledgames, datereleased, discount, price))
headers = ['date', 'game_name', 'date_released', 'discount', 'price R$']
mostselled_games = pd.DataFrame(np.array(data), columns = headers)

#### Most Selled Games DataFrame

In [55]:
mostselled_games

,date,game_name,date_released,discount,price R$
0,2020-03-23,The Witcher 3: Wild Hunt - Game of the Year Ed...,"30 Aug, 2016",70,29.99
1,2020-03-23,Grand Theft Auto V,"13 Apr, 2015",50,34.99
2,2020-03-23,Age of Empires II: Definitive Edition,"14 Nov, 2019",,36.99
3,2020-03-23,RESIDENT EVIL 3,2020/04/03,,129.99
4,2020-03-23,Assassin's Creed® Odyssey,"5 Oct, 2018",67,59.39
5,2020-03-23,Half-Life: AlyxVR Only,"23 Mar, 2020",,109.99
6,2020-03-23,Monster Hunter World: Iceborne,"9 Jan, 2020",25,67.49
7,2020-03-23,DOOM Eternal,"19 Mar, 2020",,199
8,2020-03-23,Rust,"8 Feb, 2018",50,37.74
9,2020-03-23,Pummel Party,"20 Sep, 2018",,28.99


### Getting a view of players online on steam by game

In [57]:
url = 'https://store.steampowered.com/stats/Steam-Game-and-Player-Statistics'
html = requests.get(url).content
soup = BeautifulSoup(html)
players_online = [players.text.strip().split()[0] for players in soup.find_all('tr', attrs = {'class': 'player_count_row'})]
daily_peak = [players.text.strip().split()[1] for players in soup.find_all('tr', attrs = {'class': 'player_count_row'})]
game_name = [players.find('a').text for players in soup.find_all('tr', attrs = {'class': 'player_count_row'})]

date_list = [datetime.date.today() for i in range(len(game_name))]
hour_list = [datetime.datetime.now().strftime("%H:%M:%S") for i in range(len(game_name))]
data = list(zip(date_list, hour_list, game_name, daily_peak, players_online))
headers = ['date_list', 'hour_list', 'game_name', 'daily_peak', 'players_online']
players_online = pd.DataFrame(np.array(data), columns = headers)

#### Players Online by Games DataFrame

In [58]:
players_online.head(10)

,date_list,hour_list,game_name,daily_peak,players_online
0,2020-03-23,21:30:00,Counter-Strike: Global Offensive,"1,078,859","557,327"
1,2020-03-23,21:30:00,Dota 2,"646,119","290,865"
2,2020-03-23,21:30:00,Tom Clancy's Rainbow Six Siege,"174,195","105,341"
3,2020-03-23,21:30:00,Football Manager 2020,"166,768","93,961"
4,2020-03-23,21:30:00,Grand Theft Auto V,"176,618","88,095"
5,2020-03-23,21:30:00,Team Fortress 2,"83,144","74,489"
6,2020-03-23,21:30:00,PLAYERUNKNOWN'S BATTLEGROUNDS,"573,661","70,462"
7,2020-03-23,21:30:00,Destiny 2,"82,403","70,416"
8,2020-03-23,21:30:00,Rust,"90,113","68,131"
9,2020-03-23,21:30:00,ARK: Survival Evolved,"104,788","65,442"


## Getting data from Metacritic

![Metacritic Logo](https://upload.wikimedia.org/wikipedia/commons/thumb/4/48/Metacritic_logo.svg/726px-Metacritic_logo.svg.png)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Metacritic** is a website that aggregates reviews of films, TV shows, music albums, video games and formerly, books. The site provides an excerpt from each review and hyperlinks to its source. A color of green, yellow or red summarizes the critics' recommendations. It is regarded as the foremost online review aggregation site for the video game industry.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Metacritic's** scoring converts each review into a percentage, either mathematically from the mark given, or what the site decides subjectively from a qualitative review. Before being averaged, the scores are weighted according to the critic's fame, stature, and volume of reviews. The website won two Webby Awards for excellence as an aggregation website. Criticism has focused on the assessment system, alleged third-party attempts to influence the scores, and the lack of staff oversight of user reviews.

### Creating a table of games and their notes

In [36]:
url = 'https://www.metacritic.com/browse/games/score/metascore/all/pc/filtered?page=1'

In [43]:
df = pd.DataFrame()
i = 0
status_code = 200

for i in range(2):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    url = f'https://www.metacritic.com/browse/games/score/metascore/all/pc/filtered?page={i}'
    html = requests.get(url, headers=headers).content
    soup = BeautifulSoup(html)
    game_name = [game.text.strip() for game in soup.find_all('div', attrs = {'class': 'basic_stat product_title'})]
    metacritic_score = [metascore.text for metascore in soup.find_all('div', attrs = {'class': 'metascore_w small game positive'})]
    user_score = [userscore.text.strip().replace('\n', '').split(':')[1] for userscore in soup.find_all('li', attrs = {'class': 'stat product_avguserscore'})]
    release_date = [release_date.text.strip().replace('\n', '') for release_date in soup.find_all('li', attrs = {'class': 'stat release_date full_release_date'})]
    
    data = list(zip(game_name, metacritic_score, user_score, release_date))
    headers = ['game_name', 'metacritic_score', 'user_score', 'release_date']
    metacrit = pd.DataFrame(np.array(data), columns=headers)

#### Games by Metacritic Score and Users Score

In [44]:
metacrit

,game_name,metacritic_score,user_score,release_date
0,Silent Hunter III,90,7.8,"Release Date:Mar 15, 2005"
1,Bayonetta,90,8.1,"Release Date:Apr 11, 2017"
2,DOOM Eternal,90,7.6,"Release Date:Mar 20, 2020"
3,Sid Meier's Civilization V,90,8.0,"Release Date:Sep 21, 2010"
4,Falcon 4.0: Allied Force,90,7.8,"Release Date:Jun 28, 2005"
...,...,...,...,...
95,Ori and the Blind Forest: Definitive Edition,88,8.8,"Release Date:Apr 27, 2016"
96,Shogo: Mobile Armor Division,88,8.4,"Release Date:Sep 30, 1998"
97,Thirty Flights of Loving,88,5.0,"Release Date:Aug 20, 2012"
98,Hearthstone: Heroes of Warcraft,88,5.9,"Release Date:Mar 11, 2014"


# CONNECTING PYTHON WITH POSTGRE

## Creating a function to connect and create a table

In [49]:
def create_table(title: str, df):
    engine = db.create_engine('postgresql://postgres:1fYS.9:f@localhost/games')
    conn = engine.connect()
    df.to_sql(title, con=conn, if_exists = 'replace', index=False)
    conn.close()

In [50]:
create_table(title = 'game_viewers', df = gameviewers)

In [51]:
create_table(title = 'top_channels', df = stream_channel)

In [59]:
create_table(title = 'steam_topselledgames', df = mostselled_games)

In [60]:
create_table(title = 'steam_playersonline', df = players_online)

In [13]:
import logging